<a href="https://colab.research.google.com/github/neethu-1983/Boosting-Techniques/blob/master/Datahack_AnalyticsVidhya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.metrics import mean_squared_log_error
import pandas as pd
import numpy as np
import datetime
import pickle
#from tqdm import tqdm
from scipy.stats import skew, kurtosis
import multiprocessing 
import matplotlib.pyplot as plt
import lightgbm as lgb

In [63]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive"

# downloading files from google drive

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/open?id=1Nqzhaa8PSRHJQd1a9zsfMBBnWv1R6WIF'

In [85]:
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

1Nqzhaa8PSRHJQd1a9zsfMBBnWv1R6WIF


In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
train = pd.read_csv('Filename.csv')
# Dataset is now stored in a Pandas Dataframe


In [88]:
train.tail(5)

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,User_Traffic,Competition_Metric,Sales
512082,548023,878,600,Software Marketing,Program,0,0,1,8904,0.07,114
512083,548024,879,600,Software Marketing,Program,0,0,1,10542,0.07,145
512084,548025,880,600,Software Marketing,Program,0,0,1,13671,0.07,167
512085,548026,881,600,Software Marketing,Program,0,0,1,8904,0.07,107
512086,548027,882,600,Software Marketing,Program,1,0,1,11445,0.07,152


In [0]:
from sklearn.metrics import mean_squared_log_error
def MSLE(y_true, y_pred):
  msle = 1000*np.sqrt(np.mean(np.square(np.log1p(y_true) - np.log1p(y_pred))))
  return msle



In [0]:
??mean_squared_log_error


In [0]:
link2 = 'https://drive.google.com/open?id=1wT5qr9WY1EP_msjOpUr2IoPQZkbwoYeB'

In [70]:
fluff, id = link2.split('=')
print (id) # Verify that you have everything after '='

1wT5qr9WY1EP_msjOpUr2IoPQZkbwoYeB


In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Filename.csv')  
test = pd.read_csv('Filename.csv')

In [12]:
test.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,Competition_Metric
0,883,883,1,Development,Course,1,0,1,0.007
1,884,884,1,Development,Course,1,0,1,0.007
2,885,885,1,Development,Course,1,0,1,0.007
3,886,886,1,Development,Course,1,0,1,0.007
4,887,887,1,Development,Course,0,0,1,0.007


In [13]:
train

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,User_Traffic,Competition_Metric,Sales
0,1,1,1,Development,Course,0,1,1,11004,0.007,81
1,2,2,1,Development,Course,0,0,1,13650,0.007,79
2,3,3,1,Development,Course,0,0,1,11655,0.007,75
3,4,4,1,Development,Course,0,0,1,12054,0.007,80
4,5,5,1,Development,Course,0,0,1,6804,0.007,41
...,...,...,...,...,...,...,...,...,...,...,...
512082,548023,878,600,Software Marketing,Program,0,0,1,8904,0.070,114
512083,548024,879,600,Software Marketing,Program,0,0,1,10542,0.070,145
512084,548025,880,600,Software Marketing,Program,0,0,1,13671,0.070,167
512085,548026,881,600,Software Marketing,Program,0,0,1,8904,0.070,107


In [31]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512087 entries, 0 to 512086
Data columns (total 11 columns):
ID                    512087 non-null int64
Day_No                512087 non-null int64
Course_ID             512087 non-null int64
Course_Domain         512087 non-null object
Course_Type           512087 non-null object
Short_Promotion       512087 non-null int64
Public_Holiday        512087 non-null int64
Long_Promotion        512087 non-null int64
User_Traffic          512087 non-null int64
Competition_Metric    510323 non-null float64
Sales                 512087 non-null int64
dtypes: float64(1), int64(8), object(2)
memory usage: 43.0+ MB


In [0]:
all = pd.concat([train,test],ignore_index =True)

In [77]:
all.tail(10)

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,User_Traffic,Competition_Metric,Sales
548077,548078,933,600,Software Marketing,Program,0,0,1,NaN,0.07,NaN
548078,548079,934,600,Software Marketing,Program,0,0,1,NaN,0.07,NaN
548079,548080,935,600,Software Marketing,Program,0,0,1,NaN,0.07,NaN
548080,548081,936,600,Software Marketing,Program,0,0,1,NaN,0.07,NaN
548081,548082,937,600,Software Marketing,Program,0,0,1,NaN,0.07,NaN
548082,548083,938,600,Software Marketing,Program,1,0,1,NaN,0.07,NaN
548083,548084,939,600,Software Marketing,Program,1,0,1,NaN,0.07,NaN
548084,548085,940,600,Software Marketing,Program,1,0,1,NaN,0.07,NaN
548085,548086,941,600,Software Marketing,Program,1,0,1,NaN,0.07,NaN
548086,548087,942,600,Software Marketing,Program,1,0,1,NaN,0.07,NaN


In [0]:
all['Saleslag_1'] = all.sort_values('Day_No').groupby(['Course_Domain', 'Course_Type','Course_ID'])['Sales'].shift(1)

In [0]:
#all[all.index.duplicated()]
all['Saleslag_2'] = all.sort_values('Day_No').groupby(['Course_Domain', 'Course_Type','Course_ID'])['Sales'].shift(2)

In [91]:
all

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,User_Traffic,Competition_Metric,Sales,Saleslag_1,Saleslag_2
0,1,1,1,Development,Course,0,1,1,11004.0,0.007,81.0,NaN,NaN
1,2,2,1,Development,Course,0,0,1,13650.0,0.007,79.0,81.0,NaN
2,3,3,1,Development,Course,0,0,1,11655.0,0.007,75.0,79.0,81.0
3,4,4,1,Development,Course,0,0,1,12054.0,0.007,80.0,75.0,79.0
4,5,5,1,Development,Course,0,0,1,6804.0,0.007,41.0,80.0,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
548082,548083,938,600,Software Marketing,Program,1,0,1,NaN,0.070,NaN,NaN,NaN
548083,548084,939,600,Software Marketing,Program,1,0,1,NaN,0.070,NaN,NaN,NaN
548084,548085,940,600,Software Marketing,Program,1,0,1,NaN,0.070,NaN,NaN,NaN
548085,548086,941,600,Software Marketing,Program,1,0,1,NaN,0.070,NaN,NaN,NaN


In [94]:
cutoff = all.loc[(all.index < 512087) & (all.iloc[:, 0].str.contains(r'^[4-6][A-z]*[0-9]'))].index[1]

# Using loc, not iloc, because we have the actual index
df1 = all.loc[:cutoff]
df1 = all.loc[cutoff:]

AttributeError: ignored

In [82]:
train.tail(5)

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,User_Traffic,Competition_Metric,Sales,Saleslag_1,Saleslag_2
548022,547141,938,599,Development,Program,1,0,0,NaN,0.122,NaN,NaN,NaN
548023,547142,939,599,Development,Program,1,0,0,NaN,0.122,NaN,NaN,NaN
548024,547143,940,599,Development,Program,1,0,0,NaN,0.122,NaN,NaN,NaN
548025,547144,941,599,Development,Program,1,0,0,NaN,0.122,NaN,NaN,NaN
548026,547145,942,599,Development,Program,1,0,0,NaN,0.122,NaN,NaN,NaN


In [0]:
def train_test(df, split_point):
  train_features = df.drop(['Sales'], axis = 1).head(len(df)-split_point)
  train_labels = df['Sales'].head(len(df)-split_point)
  val_features = df.drop(['Sales'], axis = 1).tail(split_point)
  val_labels = df['Sales'].tail(split_point)

  return [train_features, train_labels, val_features, val_labels]

In [0]:
train_features, train_labels, val_features, val_labels = train_test(train, 4000)

In [0]:
# def predictions_plot(model, features, labels, ax, drop_columns=['Sales']):

#   predictions = model.predict(features.drop(drop_columns, axis=1))
#   error = MSLE(labels, predictions)
#   title='MSLE:' + str(round(errors, 2)) 
        
#   pd.DataFrame({'labels': labels, 
#                 'predictions': predictions, 
#                 'Sales': features['Sales']}).plot(title=title, ax=ax, x='Sales')

# def check_predictions(model, train_features, train_labels, val_features, val_labels, drop_columns=['Sales']):
#   fig,axes  = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
#   fig.tight_layout(pad=6.0)
#   predictions_plot(model, train_features, train_labels, axes[0,0], drop_columns=drop_columns)
#   predictions_plot(model, val_features, val_labels, axes[0,1], drop_columns=drop_columns)
#   predictions_plot(model, train_features.tail(7*48), train_labels.tail(7*48), axes[1,0], drop_columns=drop_columns)
#   predictions_plot(model, val_features.head(7*48), val_labels.head(7*48), axes[1,1], drop_columns=drop_columns)

In [0]:
def predictions_plot(model, features, labels, ax):

  predictions = model.predict(features)
  error = MSLE(labels, predictions)
  title='MSLE:' + str(round(errors, 2)) 
        
  pd.DataFrame({'labels': labels, 
                'predictions': predictions, 
                }).plot(title=title, ax=ax, x='Sales')

def check_predictions(model, train_features, train_labels, val_features, val_labels):
  fig,axes  = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
  fig.tight_layout(pad=6.0)
  predictions_plot(model, train_features, train_labels, axes[0,0])
  predictions_plot(model, val_features, val_labels, axes[0,1])
  predictions_plot(model, train_features.tail(7*48), train_labels.tail(7*48), axes[1,0])
  predictions_plot(model, val_features.head(7*48), val_labels.head(7*48), axes[1,1])

In [0]:
train_features['Course_ID'] = train_features['Course_ID'].astype('category')

In [0]:
train_features['Course_Domain'] = train_features['Course_Domain'].astype('category')
train_features['Course_Type'] = train_features['Course_Type'].astype('category')

In [21]:
train_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508087 entries, 0 to 508086
Data columns (total 10 columns):
ID                    508087 non-null int64
Day_No                508087 non-null int64
Course_ID             508087 non-null category
Course_Domain         508087 non-null category
Course_Type           508087 non-null category
Short_Promotion       508087 non-null int64
Public_Holiday        508087 non-null int64
Long_Promotion        508087 non-null int64
User_Traffic          508087 non-null int64
Competition_Metric    506323 non-null float64
dtypes: category(3), float64(1), int64(6)
memory usage: 29.1 MB


In [0]:
categorical_features = ["Course_ID", "Course_Domain","Course_Type"]

In [0]:
val_features['Course_ID'] =  val_features['Course_ID'].astype('category')
val_features['Course_Domain'] =val_features['Course_Domain'].astype('category')
val_features['Course_Type'] =val_features['Course_Type'].astype('category')

In [21]:
val_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 508087 to 512086
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   ID                  4000 non-null   int64   
 1   Day_No              4000 non-null   int64   
 2   Course_ID           4000 non-null   category
 3   Course_Domain       4000 non-null   category
 4   Course_Type         4000 non-null   category
 5   Short_Promotion     4000 non-null   int64   
 6   Public_Holiday      4000 non-null   int64   
 7   Long_Promotion      4000 non-null   int64   
 8   User_Traffic        4000 non-null   int64   
 9   Competition_Metric  4000 non-null   float64 
dtypes: category(3), float64(1), int64(6)
memory usage: 231.0 KB


In [0]:
LGB_PARAMS = {'n_estimators': 10000,
                    'boosting_type': 'gbdt',
                    'objective': 'regression',
                    'metric': 'msle',
                    'subsample': 1,
                    'subsample_freq': 1,
                    'learning_rate': 0.15,
                    'feature_fraction': 1,
                    'max_depth': 4,
                    #'num_leaves': 10,
                    #'lambda_l1': 10,  
                    #'lambda_l2': 10,
                    #'early_stopping_rounds': 200,
                    'verbose': 1,
                   'random_seed': 42,
                    'categorical_feature' :   categorical_features 
                    }

In [0]:
import lightgbm as ltb

In [0]:
# train_data1 = ltb.Dataset(train_features.drop(['User_Traffic'], axis=1), label=train_labels,categorical_feature=categorical_features,free_raw_data= False)
# test_data1 = ltb.Dataset(val_features.drop(['User_Traffic'], axis=1), label=val_labels,categorical_feature=categorical_features, free_raw_data= False)

In [156]:
# %%time
# l_model_cf = ltb.train(LGB_PARAMS,
#                        train_data1,
#                        valid_sets=test_data1  , verbose_eval=False
#                         #categorical_feature=categorical_features
#                        )

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:741: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  .format(key))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


CPU times: user 6min 56s, sys: 1.8 s, total: 6min 58s
Wall time: 3min 38s


In [157]:
#  val_pred_lgb_cf = l_model_cf.predict(val_features.drop(['User_Traffic'], axis=1))

# val_pred_lgb_cf

array([ 65.07710306,  65.07710306, 130.725199  , ...,  95.71117488,
        60.48373174, 135.88413006])

In [158]:
# print (MSLE(val_labels,val_pred_lgb_cf))

439.61962475776016


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log1p
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
# if any(x<0 for x in val_pred_lgb_cf) == False:
#     print(val_pred_lgb_cf)

In [0]:
#check_predictions(l_model_cf, train_features.drop(['User_Traffic'], axis=1), train_labels, val_features.drop(['User_Traffic', 'Competition_Metric'], axis=1), val_labels)

In [0]:
# test['Course_ID'] =  test['Course_ID'].astype('category')
# test['Course_Domain'] =test['Course_Domain'].astype('category')
# test['Course_Type'] =test['Course_Type'].astype('category')

In [0]:
# test['prediction'] = l_model_cf.predict(test)

In [39]:
 pip install XlsxWriter


     |████████████████████████████████| 143kB 3.3MB/s 


In [0]:
import xlsxwriter
workbook  = xlsxwriter.Workbook('submit.xlsx')
worksheet = workbook.add_worksheet()

In [41]:
from google.colab import drive

drive.mount('/gdrive')


Mounted at /gdrive


In [0]:
# with open('/gdrive/My Drive/datahack_Brian/lightgbm.csv', 'w') as f:
#  test.to_csv(f)

Running Catboost

In [0]:
def column_index(df, query_cols):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols,query_cols,sorter=sidx)]
categorical_features_pos = column_index(train_features, categorical_features)

In [23]:
categorical_features_pos

array([2, 3, 4])

In [0]:
train_features2=train_features.copy()
val_features2=val_features.copy()

In [0]:
train_features2['Course_ID'] = train_features2['Course_ID'].astype('str')
train_features2['Course_Domain'] = train_features2['Course_Domain'].astype('str')
train_features2['Course_Type'] = train_features2['Course_Type'] .astype('str')

val_features2['Course_ID'] = val_features2['Course_ID'].astype('str')
val_features2['Course_Domain'] = val_features2['Course_Domain'] .astype('str')
val_features2['Course_Type'] = val_features2['Course_Type'].astype('str')

In [26]:
pip install catboost

     |████████████████████████████████| 64.4MB 64kB/s 


In [27]:
train_features2

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,User_Traffic,Competition_Metric
0,1,1,1,Development,Course,0,1,1,11004,0.007
1,2,2,1,Development,Course,0,0,1,13650,0.007
2,3,3,1,Development,Course,0,0,1,11655,0.007
3,4,4,1,Development,Course,0,0,1,12054,0.007
4,5,5,1,Development,Course,0,0,1,6804,0.007
...,...,...,...,...,...,...,...,...,...,...
508082,543783,406,596,Development,Program,0,0,1,12243,0.070
508083,543784,407,596,Development,Program,0,0,1,11193,0.070
508084,543785,408,596,Development,Program,0,0,1,11109,0.070
508085,543786,409,596,Development,Program,0,0,1,12327,0.070


In [0]:
import catboost as cg

In [0]:
??cg.CatBoostRegressor

In [43]:
cat_boost_cf1 = cg.CatBoostRegressor(loss_function='RMSE', subsample = 1, rsm=1, learning_rate= 0.20
                                     )#, subsample = 1,  learning_rate= 0.01, n_estimators=100)
cat_boost_cf1.fit(train_features2.drop('ID',axis=1), train_labels, cat_features=categorical_features_pos, verbose = False)

In [0]:
val_pred_ctb_cf1 = cat_boost_cf1.predict(val_features2.drop('ID', axis=1))


In [45]:
val_pred_ctb_cf1

array([ 42.16424521,  75.3518641 , 117.13007372, ..., 136.79261065,
        86.3244218 , 129.03100948])

In [0]:
#y_pred_train=model.predict(x_test)
rmsle_result = MSLE((val_labels),(val_pred_ctb_cf1)) 

In [48]:
rmsle_result

258.3956419744669

In [0]:
#train['Competition_Metric'] = round(train['Competition_Metric'] ,3)

In [0]:
train['Course_ID'] =  train['Course_ID'].astype('str')
train['Course_Domain'] =train['Course_Domain'].astype('str')
train['Course_Type'] =train['Course_Type'].astype('str')

data_pivot= pd.DataFrame(train.pivot_table(values = 'User_Traffic', index=["Course_Domain", "Course_ID","Course_Type", round(train['Competition_Metric'],3)], aggfunc='mean'))
data_pivot.fillna(0, inplace=True)


In [0]:
#train_features2['Competition_Metric']

In [202]:
data_pivot

User_Traffic
Course_Domain      Course_ID Course_Type Competition_Metric              
Business           200       Course      0.001               50190.047619
                   243       Course      0.016               39337.404762
                   431       Degree      0.011               64216.428571
                   49        Course      0.024               21352.309524
                   594       Degree      0.009               50518.547619
...                                                                   ...
Software Marketing 78        Program     0.013               11817.825215
                   81        Course      0.068               11083.047619
                   87        Course      0.034                9913.173352
                   9         Course      0.004                7868.230659
                   91        Course      0.112               14266.641834

[594 rows x 1 columns]

In [0]:
test['Course_ID'] =  test['Course_ID'].astype('str')
test['Course_Domain'] =test['Course_Domain'].astype('str')
test['Course_Type'] =test['Course_Type'].astype('str')

In [0]:
test['Competition_Metric'] = round(test['Competition_Metric'] ,3)

In [31]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36000 entries, 0 to 35999
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  36000 non-null  int64  
 1   Day_No              36000 non-null  int64  
 2   Course_ID           36000 non-null  object 
 3   Course_Domain       36000 non-null  object 
 4   Course_Type         36000 non-null  object 
 5   Short_Promotion     36000 non-null  int64  
 6   Public_Holiday      36000 non-null  int64  
 7   Long_Promotion      36000 non-null  int64  
 8   Competition_Metric  35880 non-null  float64
dtypes: float64(1), int64(5), object(3)
memory usage: 2.5+ MB


In [32]:
test2= pd.merge(test, data_pivot,'left', on = ["Course_Domain","Course_ID", "Course_Type", "Competition_Metric"])
test2

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,Competition_Metric,User_Traffic
0,883,883,1,Development,Course,1,0,1,0.007,11882.071429
1,884,884,1,Development,Course,1,0,1,0.007,11882.071429
2,885,885,1,Development,Course,1,0,1,0.007,11882.071429
3,886,886,1,Development,Course,1,0,1,0.007,11882.071429
4,887,887,1,Development,Course,0,0,1,0.007,11882.071429
...,...,...,...,...,...,...,...,...,...,...
35995,548083,938,600,Software Marketing,Program,1,0,1,0.070,8893.738095
35996,548084,939,600,Software Marketing,Program,1,0,1,0.070,8893.738095
35997,548085,940,600,Software Marketing,Program,1,0,1,0.070,8893.738095
35998,548086,941,600,Software Marketing,Program,1,0,1,0.070,8893.738095


In [36]:
#cat_boost_cf1 = cg.CatBoostRegressor(loss_function='RMSE', subsample = 1, rsm=1, learning_rate= 0.20
                                     )#, subsample = 1,  learning_rate= 0.01, n_estimators=100)
#cat_boost_cf1.fit(train.drop(['Sales','ID'], axis=1), train['Sales'], cat_features=categorical_features_pos, verbose = False)

In [0]:
test2['prediction'] = cat_boost_cf1.predict(test2.drop('ID',axis=1))

In [0]:
with open('/gdrive/My Drive/datahack_Brian/catboost.csv', 'w') as f:
 test2.to_csv(f)

Training on whole data